In [5]:
from jotform import *
import csv
import urllib
import os
from tqdm import tqdm

In [24]:
def read_dataset(api, path, filename):
    form_ids = [] 
    out = open(path + 'trimmed_' + filename, 'w+')
    writer = csv.writer(out)
    with open(path + filename, 'r') as f:
        content = f.readlines()[1:1000]
        for i, line in enumerate(tqdm(content)):
            form_id = line[:-1]
            try:
                form = api.get_form(form_id)
                writer.writerow([form_id])
                form_ids.append(form_id)
            except:
                print('form ' + form_id + ' does not exist')
    out.close()
    return form_ids

def get_form_urls(api, form_ids):
    urls = {}
    image_extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']
    for form_id in tqdm(form_ids[127:]):
        urls[form_id] = []
        for sub in api.get_form_files(form_id): 
            for ext in image_extensions:
                if sub['name'].endswith(ext):
                    if urllib.urlopen(sub['url'].encode('utf-8')).getcode() == 404:
                        if urllib.urlopen(('http://www.jotform.com/uploads/' + sub['username'] + '/form_files/' + sub['name']).encode('utf-8')).getcode() != 404:
                            urls[form_id].append('http://www.jotform.com/uploads/' + sub['username'] + '/form_files/' + sub['name'])
                    else:
                        urls[form_id].append(sub['url'])
                    break
    return urls

def create_url_dataset(path, filename, urls_dict):
    urls_file = open(path + 'urls_' + filename, 'w+')
    writer = csv.writer(urls_file, delimiter='|')
    for key in tqdm(urls_dict.keys()):
        data = [key]
        for url in urls_dict[key]:
            data.append(url.encode('utf-8'))
        if len(data) != 1:
            writer.writerow(data)
        
def create_image_dataset(path, url_dataset):
    with open(path + url_dataset) as f:
        for line in tqdm(f.readlines()[113:]):
            data = line[:-1].split('|')
            
            form_id = data[0]
            dir_name = path + 'images/' + form_id + '/'
            if not os.path.exists(dir_name):
                os.makedirs(dir_name)
            
            for img_no, url in enumerate(data[1:]):
                urllib.urlretrieve(url, dir_name + form_id + '_' + str(img_no + 1).encode('utf-8') + '.jpg')
                
def main():
    pass
    
if __name__ == "__main__":
    main()

In [7]:
api_key = '7f8ce90b8d898a20bebcb12c3d8b52be'
path = 'Desktop/jotform/ocr_tool/'
filename = 'submissionInJune408K.csv'
url_dataset = 'urls_' + filename

In [8]:
api = JotformAPIClient(api_key)
form_ids = read_dataset(api, path, filename)

100%|██████████| 999/999 [08:11<00:00,  2.03it/s]


In [18]:
urls_by_form = get_form_urls(api, form_ids)

100%|██████████| 872/872 [1:10:57<00:00,  4.88s/it]


In [19]:
create_url_dataset(path, filename, urls_by_form)

100%|██████████| 872/872 [00:00<00:00, 104704.51it/s]


In [25]:
create_image_dataset(path, url_dataset)

100%|██████████| 241/241 [26:30<00:00,  6.60s/it]
